#  Play bot (neural MCTS)

---

Author: S. Menary [sbmenary@gmail.com]

Date  : 2023-01-18, last edit 2023-01-19

Brief : Play a game of Connect 4 against bot that uses MCTS with a learned policy/value function.

---

## Imports

- Import key Python and PyPI packages and print their versions for reproducibility.
- Import the required Connect 4 game, bot and utility objects from our framework

---

In [1]:
##=====================================##
##  All imports should be placed here  ##
##=====================================##

##  Python core libs
import sys

##  PyPI libs
import numpy as np
import tensorflow as tf

##  Local packages
from connect4.utils  import DebugLevel
from connect4.game   import GameBoard
from connect4.MCTS   import PolicyStrategy
from connect4.bot    import Bot_NeuralMCTS
from connect4.neural import load_model


In [2]:
##======================================##
##  Print versions for reproducibility  ##
##======================================##

print(f"    Python version is {sys.version}")
print(f"     Numpy version is {np.__version__}")
print(f"Tensorflow version is {tf.__version__}")


    Python version is 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:29) [Clang 14.0.6 ]
     Numpy version is 1.23.2
Tensorflow version is 2.11.0


---

## Play a game vs bot

Play a game of connect 4 against our bot!

The `GameBoard` object is used as a Connect 4 game environment. It may be updated using a command like `game_board.apply_action(column_index)`. We can create a simple ASCII representation of our game board using `print(game_board)`. See `help(GameBoard)` for more useful manipulation methods.

The `Bot_NeuralMCTS` object is used to apply bot actions using neural MCTS. Call `bot.take_move(game_board, duration)` to run MCTS for `duration` seconds and then play a bot move. Turning up the `duration` parameter will improve the bot by allowing it to search for longer.

The option `PolicyStrategy.GREEDY_POSTERIOR_POLICY` commands the bot to choose the action with the highest posterior policy estimated using MCTS. To act greedily over the action values instead, use `PolicyStrategy.GREEDY_POSTERIOR_VALUES`. For a stochastic approach, use `PolicyStrategy.SAMPLE_POSTERIOR_POLICY`.

You may also skip the MCTS entirely by using `duration=0` and selecting actions using the _prior_ evaluations. In particular:
- `PolicyStrategy.GREEDY_PRIOR_POLICY` means we select the action that maximises the policy evaluated using a single neural network pass.
- `PolicyStrategy.GREEDY_PRIOR_VALUES` means we select the action that maximises the value evaluated using a single neural network pass on each of the child nodes.
- `PolicyStrategy.SAMPLE_PRIOR_POLICY` means we sample from the policy evaluated using a single neural network pass.

---

In [4]:
##=========================================##
##  Load our neural policy/value function  ##
##=========================================##

##  Configure which model to load
model_name = "../models/.neural_model_v4.h5"

##  Load this model
model = load_model(model_name)


In [5]:
##=====================##
##  Create a new game  ##
##=====================##

game_board = GameBoard()
bot        = Bot_NeuralMCTS(model, policy_strategy=PolicyStrategy.GREEDY_POSTERIOR_VALUE)
print(game_board)


+---+---+---+---+---+---+---+
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result is: NONE


---

Now play moves until the game is complete!

---

In [22]:
##===============##
##  Take moves!  ##
##===============##

##  Select our move
column_idx = 1

##  Apply our move as Player X
game_board.apply_action(column_idx)
print(game_board)

##  Apply the responding bot move move as Player O
if not game_board.get_result() :
    bot.take_move(game_board, duration=5, debug_lvl=DebugLevel.LOW)
    print(game_board)


+---+---+---+---+---+---+---+
| X | . | X | X | O | . | . |
| O | . | X | O | O | . | . |
| X | . | O | X | X | X | . |
| X | X | O | O | X | X | . |
| O | O | X | O | X | O | O |
| O | O | X | X | O | X | O |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result is: X


---

## Bot-vs-bot game

Let's watch the bot play itself!

---

In [23]:
##========================##
##  Play bot-vs-bot game  ##
##========================##


##  Set up and display game
game_board = GameBoard()
result     = game_board.get_result()
print(game_board)

##  Keep playing moves until the game concludes
while not result :
    bot.take_move(game_board, duration=5, debug_lvl=DebugLevel.LOW)
    result = game_board.get_result()
    print(game_board)


+---+---+---+---+---+---+---+
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result is: NONE
Selecting greedy action from posterior values
Action values are:  -0.654  0.038   0.036   -0.028  0.028   0.096   0.008 
Visit counts are:   1       16      12      261     11      559     2     
Selecting action 5
+---+---+---+---+---+---+---+
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | . | . |
| . | . | . | . | . | X | . |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result is: NONE
Selecting greedy action from posterior values
Action values are:  -0.142  -0.156  -0.080  -0.130  0.017   -0.290  -0.087
Visit counts are:   3 

Selecting greedy action from posterior values
Action values are:  0.169   0.046   -0.070  0.067   -0.089  -0.202
Visit counts are:   93      412     31      335     5       3     
Selecting action 0
+---+---+---+---+---+---+---+
| . | . | . | . | . | O | . |
| . | . | . | . | . | X | . |
| . | O | . | . | . | O | . |
| . | X | . | . | . | X | . |
| X | O | . | . | . | O | . |
| X | X | O | X | O | X | . |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result is: NONE
Selecting greedy action from posterior values
Action values are:  -0.212  -0.178  0.038   -0.165  -0.718  -0.572
Visit counts are:   87      16      773     19      1       3     
Selecting action 2
+---+---+---+---+---+---+---+
| . | . | . | . | . | O | . |
| . | . | . | . | . | X | . |
| . | O | . | . | . | O | . |
| . | X | . | . | . | X | . |
| X | O | O | . | . | O | . |
| X | X | O | X | O | X | . |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+--

Selecting greedy action from posterior values
Action values are:  -0.183  -0.080  -0.049  -0.194  -0.064
Visit counts are:   65      302     481     38      27    
Selecting action 3
+---+---+---+---+---+---+---+
| . | O | . | . | . | O | . |
| . | X | . | . | . | X | . |
| O | O | X | X | . | O | . |
| X | X | O | O | . | X | . |
| X | O | O | X | . | O | X |
| X | X | O | X | O | X | O |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result is: NONE
Selecting greedy action from posterior values
Action values are:  -0.808  -0.066  0.077   -0.649  -0.193
Visit counts are:   19      4       949     16      8     
Selecting action 3
+---+---+---+---+---+---+---+
| . | O | . | . | . | O | . |
| . | X | . | O | . | X | . |
| O | O | X | X | . | O | . |
| X | X | O | O | . | X | . |
| X | O | O | X | . | O | X |
| X | X | O | X | O | X | O |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result i

Selecting greedy action from posterior values
Action values are:  -0.064  -0.000  0.001 
Visit counts are:   5       10939   1945  
Selecting action 6
+---+---+---+---+---+---+---+
| X | O | . | X | . | O | . |
| O | X | O | O | . | X | O |
| O | O | X | X | X | O | X |
| X | X | O | O | X | X | O |
| X | O | O | X | O | O | X |
| X | X | O | X | O | X | O |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result is: NONE
Selecting greedy action from posterior values
Action values are:  -0.000  -0.001  0.000 
Visit counts are:   9463    179     9109  
Selecting action 6
+---+---+---+---+---+---+---+
| X | O | . | X | . | O | X |
| O | X | O | O | . | X | O |
| O | O | X | X | X | O | X |
| X | X | O | O | X | X | O |
| X | O | O | X | O | O | X |
| X | X | O | X | O | X | O |
+---+---+---+---+---+---+---+
| 0 | 1 | 2 | 3 | 4 | 5 | 6 |
+---+---+---+---+---+---+---+
Game result is: NONE
Selecting greedy action from posterior values
Action val